# PCA 2번째 과제

### 1) MNIST 데이터를 로드하고 train - testset으로 분할

In [1]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

X_train = mnist['data'][:60000]
y_train = mnist['target'][:60000]

X_test = mnist['data'][60000:]
y_test = mnist['target'][60000:]

### 2) RandomForest 분류기

#### 2-1) RF 분류기 생성  

In [2]:
from sklearn.ensemble import RandomForestClassifier # 랜포용 모듈
# 파라미터 설명: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html 

RF_classfication = RandomForestClassifier(random_state=10) # RF 분류기 생성 , random_state는 난수같은 역할 

In [3]:
import time # 시간을 나타내주는 모듈

before_fit_rf = time.time() # 훈련 전 시간 
RF_classfication.fit(X_train, y_train)
after_fit_rf = time.time() # 훈련 후 시간

print("훈련 시간 {:.2f}s".format(after_fit_rf - before_fit_rf)) # 소수점 둘째자리까지 숫자표현 

훈련 시간 3.99s


In [4]:
from sklearn.metrics import accuracy_score # 정확도 확인 모듈 

y_pred = RF_classfication.predict(X_test) # 분류기 적합 
accuracy_score(y_test, y_pred) # 94% 

0.94569999999999999

#### 2-2) PCA로 설명된 분산이 96%가 되도록 차원을 축소

In [5]:
from sklearn.decomposition import PCA # PCA 모듈 

pca = PCA(n_components=0.96) # 분산이 96%가 되도록 차원 축소
X_train_reduction = pca.fit_transform(X_train) # 축소된 train data 

#### 2-3) 축소된 데이터 셋으로 새로운 RF 분류기 예측 

In [6]:
RF_classfication2 = RandomForestClassifier(random_state=10)

before_fit2_rf = time.time()
RF_classfication2.fit(X_train_reduction, y_train)
after_fit2_rf = time.time()

print("훈련 시간 {:.2f}s".format(after_fit2_rf - before_fit2_rf)) ## 랜포는 훈련시간이 3배 느려짐 ㅠ(4.05s -> 11.93s)

훈련 시간 11.08s


In [7]:
X_test_reduction = pca.transform(X_test)

y_pred = RF_classfication2.predict(X_test_reduction)
accuracy_score(y_test, y_pred) ## 성능도 떨어짐. 이건 아니구나 (94% -> 89%) 

0.89370000000000005

### 3) Softmax 분류기 

#### 3-1) Softmax 분류기 생성  

In [8]:
from sklearn.linear_model import LogisticRegression # 로지스틱회귀용 모듈
# 파라미터 설명: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Logistic_classfication = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=10) # solver="lbfgs": 가중치^2에 패널티를 주는 L2 정규화

In [9]:
import time # 시간을 나타내주는 모듈

before_fit_sfm = time.time() # 훈련 전 시간 
Logistic_classfication.fit(X_train, y_train)
after_fit_sfm = time.time() # 훈련 후 시간

print("훈련 시간 {:.2f}s".format(after_fit_sfm - before_fit_sfm)) # 소수점 둘째자리까지 숫자표현  

훈련 시간 15.72s


In [10]:
from sklearn.metrics import accuracy_score # 정확도 확인 모듈 

y_pred = Logistic_classfication.predict(X_test) # 분류기 적합 
accuracy_score(y_test, y_pred) # 92% 

0.92520000000000002

#### 3-2) PCA로 설명된 분산이 96%가 되도록 차원을 축소

In [11]:
from sklearn.decomposition import PCA # PCA 모듈 

pca = PCA(n_components=0.96) # 분산이 96%가 되도록 차원 축소
X_train_reduction = pca.fit_transform(X_train) # 축소된 train data 

#### 3-3) 축소된 데이터 셋으로 새로운 Softmax 분류기 예측 

In [12]:
Logistic_classfication2 = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=10)

before_fit2_sfm = time.time()
Logistic_classfication2.fit(X_train_reduction, y_train)
after_fit2_sfm = time.time()

print("훈련 시간 {:.2f}s".format(after_fit2_sfm - before_fit2_sfm)) ## 오 2배 빨라짐..ㅎ (15.72s -> 7.22s)

훈련 시간 7.22s


In [13]:
X_test_reduction = pca.transform(X_test)

y_pred = Logistic_classfication2.predict(X_test_reduction)
accuracy_score(y_test, y_pred) ## 성능은 유지! (92% -> 92%) 

0.92210000000000003

### 4) 분류기 간 성능 비교

- 이 데이터는 기본 RF 분류기의 시간과 성능에 적합하다.
- PCA가 적용되고 RF 분류기는 시간/성능 모두 안좋았지만, Softmax 분류기는 시간/성능 모두 좋았다.
- 결론: 데이터마다 성능이 다르게 적용되니, 일단 데이터 받아보면 PCA는 다 해보자!!